In [ ]:
import json
import pandas
import time
import os
import csv
from functools import reduce

In [ ]:
inputDir = '..//scraped_json//'
outputDir = '..//summarized_json//'

## Load the id config
idConfig = json.loads(open('..//config//idConfig.json', "r").read())

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())
aaData = json.loads(open(inputDir + sourceFiles['allAmerican'][0], "r", encoding="utf-8").read())

In [ ]:
## Clean up dirty names
def mungeID(playerString):
    return ''.join(e for e in playerString if e.isalnum()).lower().replace("jr.", "").replace("st.", "state")  

In [ ]:
def createNewID (fieldList, thisDict, fieldAgg):
    finalID= ''
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                i[val]= mungeID(i[val][0])
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                i[val] = mungeID(i[val])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
        i['ID'] = finalID
        finalID=''

In [ ]:
def searchID(identifier, dataList):
    return [element for element in dataList if (element['ID'] == identifier)]

def search(name, team, dataList):
    return [element for element in dataList if (element['player'] == name and element['school'] == team)]

In [ ]:
createNewID(idConfig['allAmerican'], aaData, '_')
print(aaData)

[{'year': '2002', 'player': 'carsonpalmer', 'school': 'southerncalifornia', 'afca': '0', 'ap': '1', 'fwaa': '1', 'tsn': '1', 'wcff': '0', 'ID': 'carsonpalmer_southerncalifornia'}, {'year': '2002', 'player': 'kendorsey', 'school': 'miamifl', 'afca': '1', 'ap': '0', 'fwaa': '0', 'tsn': '0', 'wcff': '1', 'ID': 'kendorsey_miamifl'}, {'year': '2002', 'player': 'bradbanks', 'school': 'iowa', 'afca': '0', 'ap': '0', 'fwaa': '0', 'tsn': '0', 'wcff': '1', 'ID': 'bradbanks_iowa'}, {'year': '2002', 'player': 'byronleftwich', 'school': 'marshall', 'afca': '0', 'ap': '0', 'fwaa': '0', 'tsn': '0', 'wcff': '0', 'ID': 'byronleftwich_marshall'}, {'year': '2002', 'player': 'larryjohnson', 'school': 'pennstate', 'afca': '1', 'ap': '1', 'fwaa': '1', 'tsn': '1', 'wcff': '1', 'ID': 'larryjohnson_pennstate'}, {'year': '2002', 'player': 'willismcgahee', 'school': 'miamifl', 'afca': '0', 'ap': '1', 'fwaa': '1', 'tsn': '1', 'wcff': '1', 'ID': 'willismcgahee_miamifl'}, {'year': '2002', 'player': 'chrisbrown', 's

In [ ]:
finalOutput = []
for x in aaData:
    if (len(searchID(x['ID'], finalOutput)) == 0):
        playerList = search(x['player'], x['school'], aaData)
        finalPlayer = {}
        finalPlayer['ID'] = x['ID']
        finalPlayer['player'] = x['player']
        finalPlayer['school'] = x['school']
        year = 2021
        afca = ap = fwaa = tsn = wcff = 0
        for y in playerList:
            afca = afca + int(y['afca'])
            ap = ap + int(y['ap'])
            fwaa = fwaa + int(y['fwaa'])
            tsn = tsn + int(y['tsn'])
            wcff = wcff + int(y['wcff'])
            if (int(y['year']) < int(year)):
                year = y['year']
        finalPlayer['afca'] = afca
        finalPlayer['ap'] = ap
        finalPlayer['fwaa'] = fwaa
        finalPlayer['tsn'] = tsn
        finalPlayer['wcff'] = wcff
        finalPlayer['year'] = year
        finalOutput.append(finalPlayer)

In [ ]:
##Delete the columns I no longer care about after creating the ID
##This list will need to expand after setting up blockers and truly linking
for record in finalOutput:
    del record['year']
    del record['player']
    del record['school']
print(aaData)

[{'year': '2002', 'player': 'carsonpalmer', 'school': 'southerncalifornia', 'afca': '0', 'ap': '1', 'fwaa': '1', 'tsn': '1', 'wcff': '0', 'ID': 'carsonpalmer_southerncalifornia'}, {'year': '2002', 'player': 'kendorsey', 'school': 'miamifl', 'afca': '1', 'ap': '0', 'fwaa': '0', 'tsn': '0', 'wcff': '1', 'ID': 'kendorsey_miamifl'}, {'year': '2002', 'player': 'bradbanks', 'school': 'iowa', 'afca': '0', 'ap': '0', 'fwaa': '0', 'tsn': '0', 'wcff': '1', 'ID': 'bradbanks_iowa'}, {'year': '2002', 'player': 'byronleftwich', 'school': 'marshall', 'afca': '0', 'ap': '0', 'fwaa': '0', 'tsn': '0', 'wcff': '0', 'ID': 'byronleftwich_marshall'}, {'year': '2002', 'player': 'larryjohnson', 'school': 'pennstate', 'afca': '1', 'ap': '1', 'fwaa': '1', 'tsn': '1', 'wcff': '1', 'ID': 'larryjohnson_pennstate'}, {'year': '2002', 'player': 'willismcgahee', 'school': 'miamifl', 'afca': '0', 'ap': '1', 'fwaa': '1', 'tsn': '1', 'wcff': '1', 'ID': 'willismcgahee_miamifl'}, {'year': '2002', 'player': 'chrisbrown', 's

In [ ]:
with open(outputDir + "allAmericanData.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(finalOutput))